In [5]:
import pandas as pd
import os
#import time
import time
#import console
from py_console import console
#define location
#You can use your defined pathname as well
location = '/Users/markmulli/Desktop/Scripting/ra'
#declare a dataframe
output_dta = pd.DataFrame(columns=["L","S","V","Dept","Exit date & time","E.P","Amount(No multiplier)","Amount(With multiplier)"])
def process_data (filename):
    src = filename.split("/")[-1].split(".")[0]
    
    #current time
    console.info(src)
    #An Empty list that stores columns of a dataframe
    output_list = []
    
    curr_df = pd.read_excel(filename)
    
    for iter,rows in curr_df.iterrows():
    #create a dictionary        row_dict = {
            "Tag ID":"",
            "P":"",
            "S": "",
            "V#": "",
            "Dept": "",
            "T Date/Time": "",
            "Location": "",
            "Amount(No multiplier)": "",
            "Amount(With multiplier)": "",
        }
        
        row_dict["S"] = str(rows["P"]).split(" - ")[1].strip()
        row_dict["P"] =  str(rows["P"]).split(" - ")[0].strip()
        
        if pd.isnull(rows["Tag ID"]):
            row_dict["Tag ID"] = ""
        else:
            row_dict["Tag ID"] = str(rows["Tag ID"])
    
        
        row_dict["T Date/Time"] = (
            str(rows["T Exit Date/Time"])
            .replace(r'=TEXT("', "")
            .replace(r'","mm/dd/yyyy HH:mm:SS")', "")
            .replace(r'=Text("', "")
        )
        row_dict["Location"] = str(rows["Location"]).split(" - ")[-1]

        row_dict["Amount(No multiplier)"] = (
            str(rows["T Amount"])
            .replace("$", "")
            .replace("-", "")
            .replace("(", "")
            .replace(")", "")
        )
        row_dict["Amount(With multiplier)"] = (  
            str(rows["T Amount"])  
            .replace("$", "")
            .replace("-", "")
            .replace("(", "")
            .replace(")", "")
        )
        output_list.append(row_dict)
    
    df = pd.DataFrame(output_list)
        
    df = df.astype({"Amount(With multiplier)":"float64","Amount(No multiplier)":"float64","T Date/Time":"datetime64[ns]"})
        
    return df
    
#start time
start = time.time()
final_df = output_dta

unprocessed = []
#list directory method for accessing the directory for operations to be performed on them
for i in os.listdir(location):
    if i.lower().endswith(".xlsx"):
        if i.startswith("filename"):
            print(i)
            final_df = pd.concat([final_df,process_data(location +"/"+ i)],ignore_index=True)
            print("!! processed :" + i)
        else:
            unprocessed.append(i)
    else:
        unprocessed.append(i)
#save file
datatoexcel = pd.ExcelWriter(f'output_files/modified.xlsx',engine="xlsxwriter")

final_df[["Amount(With multiplier)"]] = final_df[["Amount(With multiplier)"]]*1.41

final_df.to_excel(datatoexcel,index=False)

datatoexcel.close()

#stop time
stop = time.time()
#output time
console.success(f'Done processing in {stop-start}')
            
    